In [164]:
%matplotlib inline
import pylab as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import Normalizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from NNeighbour import NearestNeighbor

In [5]:
path = 'https://raw.githubusercontent.com/Codecademy/datasets/master/streeteasy/queens.csv'
df = pd.read_csv(path)

In [67]:
abs(df.corr()['rent']).sort_values()

has_washer_dryer    0.045068
has_dishwasher      0.057593
has_doorman         0.068435
has_patio           0.074720
min_to_subway       0.076548
has_roofdeck        0.106541
has_gym             0.132922
has_elevator        0.156086
rental_id           0.193360
no_fee              0.284298
floor               0.387964
building_age_yrs    0.417359
bedrooms            0.485529
size_sqft           0.548668
bathrooms           0.559759
rent                1.000000
Name: rent, dtype: float64

In [183]:
columns = ('no_fee', 'has_roofdeck', 'has_washer_dryer', 'has_doorman', 
           'has_elevator', 'has_dishwasher', 'has_patio', 'has_gym')
useless = ['rent', 'rental_id', 'min_to_subway', 'has_washer_dryer', 'has_dishwasher',
          'has_doorman', 'has_patio', 'has_roofdeck']

In [10]:
for el in columns:
    df[el] = df[el] * 2 - 1

In [79]:
df.head(5)

,rental_id,rent,bedrooms,bathrooms,size_sqft,min_to_subway,floor,building_age_yrs,no_fee,has_roofdeck,has_washer_dryer,has_doorman,has_elevator,has_dishwasher,has_patio,has_gym,neighborhood,borough
0,10234,3000,3.0,1,1000,4,1.0,106,-1,-1,-1,-1,-1,-1,-1,-1,Astoria,Queens
1,10524,1950,1.0,1,950,1,3.0,83,-1,-1,-1,-1,-1,-1,-1,-1,Forest Hills,Queens
2,5465,3500,1.0,1,996,9,7.0,3,1,-1,-1,-1,-1,-1,-1,-1,Astoria,Queens
3,5259,1725,1.0,1,800,1,2.0,89,-1,-1,-1,-1,-1,-1,-1,-1,Astoria,Queens
4,9736,1700,1.0,1,730,10,3.0,7,-1,-1,-1,-1,-1,-1,-1,-1,Briarwood,Queens


In [184]:
N = Normalizer()
y = df.rent
X = df.drop(columns = useless, axis=1).select_dtypes('number')

In [219]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)
N.fit(X_train)

Normalizer(copy=True, norm='l2')

In [220]:
X_train_sc =  pd.DataFrame(N.transform(X_train), index=X_train.index, columns=X_train.columns) 
X_test_sc = pd.DataFrame(N.transform(X_test), index=X_test.index, columns=X_test.columns)

## Trying to implement self-written KNN

In [187]:
model = NearestNeighbor()
modelsc = NearestNeighbor()

In [188]:
model.fit(X_train, y_train)
modelsc.fit(X_train_sc, y_train)

In [189]:
y_pred = model.predict(X_test)
y_pred_sc = modelsc.predict(X_test_sc)

In [190]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
for metric in  mean_absolute_error, mean_squared_error, r2_score:
     print(f'{metric.__name__}: {metric(y_pred, y_test)}')

mean_absolute_error: 400.74444444444447
mean_squared_error: 423713.47777777776
r2_score: 0.29820471089346134


In [191]:
for metric in  mean_absolute_error, mean_squared_error, r2_score:
     print(f'{metric.__name__}: {metric(y_pred_sc, y_test)}')

mean_absolute_error: 492.1333333333333
mean_squared_error: 620316.8222222222
r2_score: 0.08761835838921872


## Trying to implement sklearn's KNN

In [223]:
from sklearn.neighbors import KNeighborsClassifier
error = []
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X_train_sc, y_train) 
y_pred_b = neigh.predict(X_test_sc)
error.append(mean_absolute_error(y_pred_b, y_test))

In [224]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
for metric in  mean_absolute_error, mean_squared_error, r2_score:
     print(f'{metric.__name__}: {metric(y_pred_b, y_test)}')

mean_absolute_error: 546.2111111111111
mean_squared_error: 670561.6777777778
r2_score: -0.7795006143070637


1) Лучший результат достигается при n_neighbor = 2
2) Отбросить фичи, которые особо не влияют на результат (корреляция незначительная), нормализовать данные, почистить датасет
3) Чем больше фич, тем больше времени требуется
4) Недостатки написанной нами KNN:
    - невозможность выбора количества соседей
    - отсутсвие метода predict_proba (soft_voting невозможен)
   Недостатки KNN в целом:
    - сильно зависит от количества фич
    - большое значение придается выбросам
    - из-за медленности работы бывает трудно найти оптимальное количество соседей
    - не умеет работать с NaN значениями